# Daily averages with filter
This script calculates the daily average of multiple tags, accounting for filtered periods. The script follows these steps:
1. Filtered-out periods are obtained from a search
2. Full days are obtained from a search (day tag = constant, no filter)
3. Partial filtered results within each day are obtained (day tag = constant, excluded filtered periods)
4. For each day, take the weighthed average of filtered result averages to get a single global average
    - Partial results are weighted by their duration
5. Export results to csv

## 0a. Settings
Imports and settings to the the script. Besides these settings the value-based searches also need to be configured down in the code.

In [1]:
import numpy as np
import pandas as pd
from trendminer_interface import TrendMinerClient


search_period = "1y"

time_tag = "TM_day_Europe_Brussels"

# Only average (mean) calculations will work for this script!
calculations = {
    "temp in": ("TM-T200-TI201.PV", "mean", "°C"),
    "temp out": ("TM-T200-TI202.PV", "mean", "°C")
    }

timezone = "Europe/Brussels"

filename = "calculations.csv"

## 0b. Authentication
Log in to the server with client id and client secret. The client id used should have been given access rights to the date in ConfigHub > Access.

Rather than use the keyring package to get a locally stored client secret, the client secret can be given as a string.

In [2]:
import keyring

url = "https://demo.trendminer.cloud"
client_id = "wdanielsclient"
username = "wdaniels"

client_secret = keyring.get_password(url, client_id)
password = keyring.get_password(url, username)

client = TrendMinerClient(
    url=url, 
    client_id=client_id, 
    client_secret=client_secret,
    tz = timezone,
)

## 1. Filtered interval retrieval
We get the intervals we want to exclude from our search using a value-based search. No duration is given (defaults to 2x the index resolution).

In [3]:
vbs = client.search.value(
    queries = [
        ("TM-T200-FI201.PV", "<", 7)
    ]
)

excluded_intervals = vbs.get_results(search_period)

## 2. Full interval retrieval
We get the intervals we want to calculate our average on. We don't calculate the average on these intervals directly, since they will contain intervals that need to be filtered out.

In [4]:
vbs = client.search.value(
    queries = [
        (time_tag, "constant")
    ],
    duration = "23h"
)

original_intervals = vbs.get_results(search_period)

## 3. Subintervals retrieval and calculation
These are the filtered-in periods that we actually want to do the calculations on. We obtain the filtered-in periods by again searching on our time tag, but this time we exclude the intervals obtained in step 1 in the search execution.

In [5]:
vbs = client.search.value(
    queries = [
        (time_tag, "constant")
    ],
    calculations=calculations
)

results = vbs.get_results(search_period, excluded_intervals=excluded_intervals)

## 4. Calculation on full intervals
For every full interval retrieved in step 2, we can now determine what subintervals fall in the full interval, and calculate a global value from these. The total average value can be found bij taking the averages of the subintervals weighted by their duration.

In the calculation, we must account for the scenario that there might not be any subintervals in the full interval (i.e., the entire full interval was filtered out by the criteria in step 1).

In [6]:
for interval in original_intervals:
    
    # Get results that fall in this interval
    results_in_interval = [
        result for result in results if result.start >= interval.start and result.end <= interval.end
    ]
    
    total_duration = sum([result.total_seconds() for result in results_in_interval])
    
    # Calculate averages over all results
    for key in calculations.keys():
        if total_duration == 0:
            avg = np.nan
        else:
            avg = sum([result.data[key]*result.total_seconds()/total_duration for result in results_in_interval])
        interval.data.update({key: avg})

## 5. Export to CSV
We can create a DataFrame from the data we added to full intervals. We add the interval start time as the index for the calculation. For clarity, we reduce the timestamps to single dates, which is fine if the interval length is a number of days.

In [7]:
# to dataframe
df = pd.DataFrame.from_records(
    [interval.data for interval in original_intervals],
    index=[interval.start.date() for interval in original_intervals]
)

# sort by ascending date
df.sort_index(inplace=True)

# save to csv
df.to_csv(filename)

df

,temp in,temp out
2021-12-24,20.714638,44.641464
2021-12-25,20.734539,45.049110
2021-12-26,20.729208,44.956966
2021-12-27,20.741962,45.136936
2021-12-28,20.230228,45.103092
...,...,...
2022-12-18,20.587900,44.930430
2022-12-19,20.652626,45.067860
2022-12-20,20.938768,44.908806
2022-12-21,20.156928,45.221926
